In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
import time
import pandas as pd
import warnings
import pymysql.cursors
from sqlalchemy import create_engine
import re
import requests
import random
warnings.filterwarnings('ignore')

In [2]:
# db connect get_movie_info
user = "chipnday"
password = "chipnday2022"
host = "132.226.150.234:3306"
db = "chipnday_db"
table = 'MovieInfo'
db_connection_str = f'mysql+pymysql://{user}:{password}@{host}/{db}'
encoding = '?charset="utf8", encoding="utf-8"'
db_connection = create_engine(db_connection_str)
conn = db_connection.connect()
from_db_df = pd.read_sql_table(table, db_connection)
from_db_dicts = []
for i in range(len(from_db_df)):
    from_db_dict = {'skeword':from_db_df['TitleEn'].values.tolist()[i],'KinoId':from_db_df['KinoId'].values.tolist()[i]}
    from_db_dicts.append(from_db_dict)
# ""제거
newtitles = []
for from_db in from_db_dicts:
    if from_db['skeword']=="":
        pass
    else:
        newtitles.append(from_db)

#newtitles.pop(0)
len(newtitles)

1

In [3]:
# 4 중복 제거 get img_url
# db connect
user = "chipnday"
password = "chipnday2022"
host = "132.226.150.234:3306"
db = "chipnday_db"
table = 'ImgUrl'
db_connection_str = f'mysql+pymysql://{user}:{password}@{host}/{db}'
encoding = '?charset="utf8", encoding="utf-8"'
db_connection = create_engine(db_connection_str)
conn = db_connection.connect()
from_db_df = pd.read_sql_table(table, db_connection)
old_url_list =  from_db_df['KinoId'].values.tolist()

In [4]:
len(old_url_list)

0

In [5]:
final_dicts = []
for i in range(len(newtitles)):
    if  not (str(newtitles[i]['KinoId']) in old_url_list ):
        final_dicts.append(newtitles[i])
len(final_dicts)

1

In [6]:
# 사이트 접속
driver = webdriver.Chrome()
driver.get("https://www.imdb.com/")
time.sleep(0.3) #1초 웨이팅 없으면 리젝당할수도...&창꺼짐

In [7]:
# 영화 정보 창 ㄱㄱ
def search(skeword):
    elem = driver.find_element_by_name('q')
    elem.send_keys(skeword)
    elem.submit()
    time.sleep(0.3)
    driver.find_element_by_xpath('//*[@id="main"]/div/div[2]/table/tbody/tr[1]/td[2]/a').click()
    time.sleep(1)

In [8]:
def less_img(kino_id,skeword):
    img_df =  pd.DataFrame(columns = [ 'KinoId', 'TitleEn', 'SmallImg'])
    cnt = 0
    img_dicts = []
    try:
        while True:
            cnt += 1
            get = driver.find_elements_by_css_selector(f'.ipc-shoveler__grid > div:nth-child({cnt}) > div > img')
            url = get[0].get_attribute('src')
            img_dict = {'KinoId':kino_id, 'TitleEn':skeword, 'SmallImg':url}
            img_df = img_df.append(img_dict, ignore_index=True)
    except IndexError as ie:
        print(f'{cnt}개의 이미지 가져옴:{ie}')

    return img_df

In [10]:
from selenium.common.exceptions import NoSuchElementException
import db
for i in range(len(final_dicts)):
    try:
        skeword = final_dicts[i]['skeword']
        kino_id = final_dicts[i]['KinoId']
        search(skeword)
        imgs_df = less_img(kino_id,skeword)
        db.db_insert(imgs_df,'ImgUrl')
        print(f'{i+1} / {len(final_dicts)}')
    except NoSuchElementException as NE:
        print(f'{NE}: No Result')
        pass
    except all as err:
        print(err)
        pass
    # break
print(f'컨텐츠{i+1}개의 이미지 크롤링 완료')

13개의 이미지 가져옴:list index out of range
12 insert success
1 / 1
컨텐츠1개의 이미지 크롤링 완료


### 추후구현 - 이미지 리사이즈로 대채 더많은 이미지 필요시 개발

In [228]:
#추후 구현
def lot_img(kinoid,skeword):
    img_df =  pd.DataFrame(columns = [ 'KINO_ID', 'TITLE_EN', 'URL'])
    img_dicts = []
    cnt = 1
    while True:
        cnt += 1
        try:
            more_img = driver.find_elements_by_xpath(f'//*[@id="media_index_thumbnail_grid"]/a[{cnt}]/img')[0]
            url = more_img.get_attribute('src')
            img_dict = {'KINO_ID':kinoid, 'TITLE_EN':skeword, 'URL':url}
            img_df = img_df.append(img_dict, ignore_index=True)
        except IndexError as ie:
            print(f'get lot img err: {ie}')
            break
    return img_df

# 이미지 리사이즈

In [2]:
# from db
# db connect
user = "chipnday"
password = "chipnday2022"
host = "132.226.150.234:3306"
db = "chipnday_db"
table = 'ImgUrl'
db_connection_str = f'mysql+pymysql://{user}:{password}@{host}/{db}'
encoding = '?charset="utf8", encoding="utf-8"'
db_connection = create_engine(db_connection_str)
conn = db_connection.connect()
from_db_df = pd.read_sql_table(table, db_connection)
from_db_df.head(3)

,KinoId,TitleEn,SmallImg,OgImg
0,13908,Avatar,https://m.media-amazon.com/images/M/MV5BMjEyOT...,None
1,13908,Avatar,https://m.media-amazon.com/images/M/MV5BMjEyOT...,https://m.media-amazon.com/images/M/MV5BMjEyOT...
2,13908,Avatar,https://m.media-amazon.com/images/M/MV5BNzM2MD...,https://m.media-amazon.com/images/M/MV5BNzM2MD...


In [3]:
import db

old_urls = from_db_df['SmallImg'].values.tolist()
print(len(old_urls))
new_urls = []
for url in old_urls:
    new_urls.append(url.split("V1")[0])
from_db_df['OgImg'] = new_urls
db.db_insert(from_db_df,'ImgUrl')

13
12 insert success


### re 정규식 가용

In [ ]:
pattern = "100"
pattern = "QL([0-9]{3}|[0-9]{2}|[0-9])_UX([0-9]{3}|[0-9]{2}|[0-9])_CR([0-9]{3}|[0-9]{2}|[0-9]),([0-9]{3}|[0-9]{2}|[0-9]),([0-9]{3}|[0-9]{2}|[0-9]),([0-9]{3}|[0-9]{2}|[0-9])"
pattern = "UY100_CR[0-9]{2},([0-9]{2}|[0-9]),100,100"
replace = ""
resize_url = re.sub(pattern, replace, url)
img_dicts